## Comments about Assignment 1
#### Adriano Mundo 10524163 , Mario Sacaj 10521887
Assignment 1 for the ANN2DL course was to participate a competition on kaggle with the objective to develop a Neural Network Architecture for Image Classification.

We, as showed us in the practice classes, developed a Convolutional Neural Network for the classification task.

We started with the architecture provided by the professor trying to understand what every piece of code do, then we re-elaborate our architecture, and instead of using the class structure, we decided to use the Sequential architecture, so it was easier to test, adding and removing new layers of the network. 

In order to divide our training set in train and validation we used the ImageDataGenerator combined with the SEED to make the experiment reproducible with a 80/20 split. 

Then, we started experimenting with data augmentation in order to improve the network. Analysing our image set we noticed that image are larger than what we expect, so we increased the image shape, and because of our limited amounts of data, we played with the ImageDataGenerator to increase our train dataset, for example adding rotation, width and heigh shift range, brightness range, etc. 

In order to deal with overfitting, between the two Dense Layer we added a Dropout with 0.5 as a parameter.

Then we trained our network using relu as activation function for all the network expect the last layer were we used softmax. The metric, as stated by the task was accuracy, while loss function and optimzer are respectively categorical cross entropy and adam. 

We obtained a 64% of accuracy, we did other experiments and research and we can say that it's possible to reach about a maxium of 70%  accuracy with these kind of architecture trying to tune again the parameter to find the match. 

We decided to not use Transfer Learning because it is very easy to conquer this task using a pre-trained network, vgg to obtain an accuracy of more or less 100%. We, as reported in the task assignment ,tried to understand the concepts, what does it make sense to do and what does not make sense without thinking about obtaining the best result because it was very easy to implement that kind of architecture. 


In [10]:
import os
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from datetime import datetime
import json

# this let the experiment to be reproducible
SEED = 1234
tf.random.set_seed(1234)
cwd = os.getcwd()

#Dataset directory
dataset_dir = "/kaggle/input/ann-and-dl-image-classification/Classification_Dataset"

#Model weights filepath (set to None if you wanna train your model from scratch)
WEIGHTS = '/kaggle/input/model4me/model.h5'
weights = None

#Create Dataset_split.json for competition purposes (set to None if you don't want to create it)
dataset_info = None


#Definition of results writedown function
def create_csv(results, results_dir=''):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [11]:

# batch size
bs = 8

# img shape
img_h = 320
img_w = 320

# ImageDataGenerator
# -------------------------------------------------------
# data augmentation to improve performances
apply_data_augmentation = True

# create training/validation ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=20,
                                        width_shift_range=20,
                                        height_shift_range=20,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        brightness_range = [0.5, 1.5],
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255,
                                        validation_split = 0.2)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# create testing ImageDataGenerator object
test_data_gen = ImageDataGenerator(rescale=1./255)
# -------------------------------------------------------


In [12]:

# list of classes
num_classes = 20
decide_class_indices = True
if decide_class_indices:
    classes = ['owl',                   # 0
               'galaxy',                # 1
               'lightning',             # 2
               'wine-bottle',           # 3
               't-shirt',               # 4
               'waterfall',             # 5
               'sword',                 # 6
               'school-bus',            # 7
               'calculator',            # 8
               'sheet-music',           # 9
               'airplanes',             # 10
               'lightbulb',             # 11
               'skyscraper',            # 12
               'mountain-bike',         # 13
               'fireworks',             # 14
               'computer-monitor',      # 15
               'bear',                  # 16
               'grand-piano',           # 17
               'kangaroo',              # 18
               'laptop']                # 19

else:
    classes = None

In [13]:
# Generators
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               color_mode='rgb',
                                               classes=classes,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED, subset='training')


valid_gen = train_data_gen.flow_from_directory(training_dir,
                                               color_mode='rgb',
                                               batch_size=bs,
                                               classes=classes,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED, subset='validation')

test_gen = test_data_gen.flow_from_directory(dataset_dir,
                                             classes=['test'],
                                             color_mode='rgb',
                                             target_size=(img_h, img_w),
                                             class_mode=None,
                                             shuffle=False,
                                             seed=SEED)
# -------------------------------------------------------

Found 1247 images belonging to 20 classes.
Found 307 images belonging to 20 classes.
Found 500 images belonging to 1 classes.


In [14]:
# Create Dataset objects
# -------------------------------------------------------
# training and repeat
#train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
 #                                              output_types=(tf.float32, tf.float32),
  #                                             output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
#train_dataset = train_dataset.repeat()

# validation and repeat
#valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               #output_types=(tf.float32, tf.float32),
                                               #output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
#valid_dataset = valid_dataset.repeat()
# -------------------------------------------------------

In [15]:
# Neural Network Architecture with Sequential model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=8,kernel_size=(3, 3),strides=(1, 1), padding='same', activation = 'relu', input_shape=(img_h, img_w, 3)))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(filters=16,kernel_size=(3, 3),strides=(1, 1),padding='same', activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(3, 3),strides=(1, 1),padding='same', activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(filters=64,kernel_size=(3, 3),strides=(1, 1),padding='same', activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))         
model.add(tf.keras.layers.Conv2D(filters=128,kernel_size=(3, 3),strides=(1, 1),padding='same', activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))   
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [16]:
# Optimization parameters
# -------------------------------------------------------
# loss
loss = tf.keras.losses.CategoricalCrossentropy()
# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# validation metrics
metrics = ['accuracy']
# compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
# -------------------------------------------------------

In [17]:
# Optional model_weights loading from file
if weights is not None:
    model.load_weights(weights)

In [18]:
# Training
if weights is None:
    
    #Training with callbacks
    #-------------------------------------------------------
    callbacks = []

    # Early Stopping
    early_stop = True
    if early_stop:
        es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10) #, restore_best_weights=True)
        callbacks.append(es_callback)
    model.fit_generator(train_gen, steps_per_epoch=len(train_gen), epochs=100, callbacks=callbacks, validation_data=valid_gen, validation_steps=len(valid_gen))


Epoch 1/100
156/156 [==============================] - 38s 245ms/step - loss: 2.9863 - accuracy: 0.0682 - val_loss: 2.9504 - val_accuracy: 0.1010
Epoch 2/100
156/156 [==============================] - 37s 236ms/step - loss: 2.9074 - accuracy: 0.0986 - val_loss: 2.8330 - val_accuracy: 0.1270
Epoch 3/100
156/156 [==============================] - 36s 232ms/step - loss: 2.7857 - accuracy: 0.1443 - val_loss: 2.6472 - val_accuracy: 0.1954
Epoch 4/100
156/156 [==============================] - 36s 234ms/step - loss: 2.6055 - accuracy: 0.1740 - val_loss: 2.5696 - val_accuracy: 0.2117
Epoch 5/100
156/156 [==============================] - 36s 231ms/step - loss: 2.4292 - accuracy: 0.2654 - val_loss: 2.4168 - val_accuracy: 0.2541
Epoch 6/100
156/156 [==============================] - 36s 231ms/step - loss: 2.3218 - accuracy: 0.3031 - val_loss: 2.2454 - val_accuracy: 0.2997
Epoch 7/100
156/156 [==============================] - 36s 232ms/step - loss: 2.1871 - accuracy: 0.3448 - val_loss: 2.1390 -

In [19]:
#Compute Predictions
test_gen.reset()
preds = model.predict_generator(test_gen)
preds_cls_idx = preds.argmax(axis=-1)

In [21]:
# Write down results file
image_filenames = test_gen.filenames
image_filenames
results = {}
for i in range(len(image_filenames)):
   results[(image_filenames[i][5:])] = preds_cls_idx[i]
create_csv(results)

In [ ]:
# Create dictionary for the .json file 
if dataset_info is not None:
    from collections import defaultdict
    train_dict = defaultdict(list)
    for train_image in train_gen.filenames:
        clss = train_image.split('/')
        key = clss[0]
        val = clss[1]
        train_dict[key].append(val)

    valid_dict = defaultdict(list)
    for valid_image in valid_gen.filenames:
        clss = valid_image.split('/')
        key = clss[0]
        val = clss[1]
        valid_dict[key].append(val)

    master_dict = {}
    master_dict['training'] = train_dict
    master_dict['validation'] = valid_dict

    with open('dataset_split.json', 'w') as json_file:
      json.dump(master_dict, json_file)